In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster
client = Client(n_workers=1,
                threads_per_worker=4,
                memory_limit='16GB')
client

In [ ]:
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da
import time
import os

import dask

import matplotlib.pyplot as plt
import hvplot.xarray
import holoviews as hv
import scipy.constants
import scipy

sys.path.append("../..")
import processing_dask as pr
import plot_dask

sys.path.append("../../../preprocessing/")
from generate_chirp import generate_chirp

In [ ]:
#prefix = "/media/thomas/Extreme SSD/orca_paper_data_files/phase_noise/b205/20240222_203345"

#prefix = "/Volumes/Extreme SSD/orca_paper/20240226_105437" # no phase dithering, no LO offset
#prefix = "/Volumes/Extreme SSD/orca_paper/20240226_105916" # yes phase dithering, no LO offset
#prefix = "/Volumes/Extreme SSD/orca_paper/20240226_110410" # no phase dithering, software LO offset of 12.5 MHz
#prefix = "/Volumes/Extreme SSD/orca_paper/20240226_110948" # yes phase dithering, software LO offset of 12.5 MHz

prefix = "/Volumes/Extreme SSD/orca_paper_data_files/dithering/b205/20240226_225223" # no phase dithering, software LO offset of 12.5 MHz

#zero_sample_idx = 63 # X310, fs = 50 MHz
zero_sample_idx = 159

dielectric_constant = 2.2957 # ice (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

zarr_path = pr.save_radar_data_to_zarr(prefix)

zarr_path

In [ ]:
raw = xr.open_zarr(zarr_path)

In [ ]:
chirp_ts, chirp = generate_chirp(raw.config)

compressed = pr.pulse_compress(raw, chirp,
                               fs=raw.config['GENERATE']['sample_rate'],
                               zero_sample_idx=zero_sample_idx,
                               signal_speed=sig_speed).persist()

In [ ]:
# save pulse compressed data to location
#zarr_base_location = "/Volumes/Extreme SSD/orca_paper/"
zarr_base_location = "/Volumes/Extreme SSD/orca_paper_data_files/dithering/b205"
compressed_zarr_path = os.path.join(zarr_base_location, raw.basename + "_pulsecompressed.zarr")
print("Writing pulse compressed data to: ", compressed_zarr_path)

compressed.to_zarr(compressed_zarr_path, mode='w')

In [ ]:
compressed = xr.open_zarr("/Volumes/Extreme SSD/orca_paper_data_files/dithering/b205/20240226_225223_pulsecompressed.zarr")

In [ ]:
stacks = [1, 10, 100, 1000, 10000, 100000, 1000000]
ts = stacks

noise_start_distance_1way = 1000 # m

In [ ]:
compressed

## Noise Floor Variance

In [ ]:
actual_stack_t = np.nan * np.zeros_like(ts)
actual_stack_n = np.zeros_like(ts, dtype=int)

# Statistics to compute
stack_noise_var = np.nan * np.zeros_like(ts)
stack_noise_mean = np.nan * np.zeros_like(ts)

In [ ]:
for t_idx, t in enumerate(ts):
    if not np.isnan(stack_noise_mean[t_idx]):
        continue # Skip if already computed (in case of interruption and restart)
    
    timestamp = time.time() # Track computation time 

    actual_stack_n[t_idx] = t
    actual_stack_t[t_idx] = actual_stack_n[t_idx] * raw.attrs['config']['CHIRP']['pulse_rep_int'] # TODO: Account for errors?
    print(f"[{t_idx+1}/{len(ts)}] \tt={actual_stack_t[t_idx]} \tn_stack={actual_stack_n[t_idx]}")
    
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):

        stacked = pr.stack(compressed, actual_stack_n[t_idx])
        compressed_pwr = xr.apply_ufunc(lambda x: np.abs(x)**2, stacked, dask='parallelized').chunk("auto")
        
        noise_pwr = compressed_pwr["radar_data"].where((compressed_pwr.reflection_distance > noise_start_distance_1way)).dropna('travel_time').chunk("auto")
        
        stack_noise_var[t_idx] = noise_pwr.var(dim="travel_time").mean().compute().item()
        stack_noise_mean[t_idx] = noise_pwr.mean().compute().item()

        
    print(f"Completed in {time.time() - timestamp} seconds from {len(noise_pwr)} stacked pulses")

In [ ]:
fig, (ax_noise_var, ax_noise_mean) = plt.subplots(2, 1, figsize=(10, 20))

ax_noise_var.scatter(actual_stack_n, stack_noise_var)
ax_noise_var.set_title("Noise Variance")
ax_noise_var.loglog()
ax_noise_var.grid()

ax_noise_mean.scatter(actual_stack_n, stack_noise_mean)
ax_noise_mean.set_title("Noise Mean")
ax_noise_mean.loglog()
ax_noise_mean.grid()

In [ ]:
import pickle
with open(f"outputs/{raw.basename}-noise-stats.pickle", "wb") as f:
    pickle.dump({'n_stacks': actual_stack_n, 'stack_times': actual_stack_t, 'stack_noise_var': stack_noise_var, 'stack_noise_mean': stack_noise_mean, 'prefix': raw.prefix}, f)